In [ ]:
import pandas as pd

data = pd.read_csv('./data/train.csv')

data.drop(labels=['author', 'license', 'url', 'collection', 'common_name'], axis=1, inplace=True)

In [ ]:
import os

folder_dict = os.listdir('./data/train_audio')

normalized_labels = dict()
label_idx = 0
for label in data['primary_label']:
    if label in normalized_labels:
        continue
    else:
        normalized_labels[label] = label_idx
        label_idx += 1

assert len(normalized_labels.keys()) == len(folder_dict)

data['primary_label'] = data['primary_label'].map(lambda x: normalized_labels[x])

label_to_name = dict()

for idx, label in enumerate(data['primary_label']):
    try:
        name = data[data['primary_label'] == label]['scientific_name'][idx]
        if label in label_to_name:
            continue
        label_to_name[label] = name
    except:
        print(label)

label_to_name




In [ ]:
#data.drop('scientific_name', axis=1, inplace=True)
data


In [ ]:
label_to_name

In [ ]:
import json
with open('label_to_name.json', 'w') as file:
    json.dump(label_to_name, file)

data.to_csv('processed_data.csv', index=False)

In [1]:
highest_idx = 0

In [4]:
import torchaudio
import torch
import os

CHUNK_SIZE = 5
STRIDE = 2.5
chunk_amount = int(32000*CHUNK_SIZE)
stride_amount = int(32000*STRIDE)
melspectogram = torchaudio.transforms.MelSpectrogram()
amptodb = torchaudio.transforms.AmplitudeToDB()

for label in os.listdir('./data/train_audio'):
    os.makedirs(f'./data/processed_train_audio/{label}', exist_ok=True)
    for file in os.listdir(f'./data/train_audio/{label}'):
        audio_file, sr = torchaudio.load(f'./data/train_audio/{label}/{file}')        
        total_len = audio_file.shape[1]
        chunks = []
        for i in range(0, total_len - chunk_amount + 1, stride_amount):
            chunk = audio_file[:, i:i+chunk_amount]
            mel = melspectogram(chunk)
            processed_chunk = amptodb(mel)
            chunks.append(processed_chunk)
        if chunks:
            stacked_chunks = torch.stack(chunks)
            torch.save(stacked_chunks, f'./data/processed_train_audio/{label}/{file[:-4]}.pt')



/home/maxim/miniforge3/envs/nlp/lib/python3.12/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [5]:
del processed_chunk_list

In [3]:
import torch
torch.load('./data/processed_train_audio/21116.pt').shape

torch.Size([2, 1, 128, 313])

# Deprecated

In [ ]:
import torchaudio
import os
import tqdm
import torch
import gc

CHUNK_LENGTH=5
STRIDE=2.5

mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=32000, n_fft=1024)
amp_transform = torchaudio.transforms.AmplitudeToDB()


def process_chunk(label_path, label):
    chunks = []
    for file in os.listdir(label_path):
        path = f"{label_path}/{file}"
        audio_file, sr = torchaudio.load(path)
        total_len = audio_file.shape[1]
        chunk_amount = int(sr * CHUNK_LENGTH)
        stride_amount = int(sr*STRIDE)
        if total_len < chunk_amount:
            continue
        for i in range(0, total_len - chunk_amount + 1, stride_amount):
            chunk = audio_file[:, i:i+chunk_amount]

            mel = mel_transform(chunk)
            processed_chunk = amp_transform(mel)
            chunks.append(processed_chunk)
    if chunks:
        torch.save(torch.stack(chunks), f"./data/processed_train_audio/{label}.pt")
        del chunks, mel, processed_chunk, audio_file
        gc.collect()



for label in tqdm.tqdm(os.listdir('./data/train_soundscapes')):
    process_chunk(f'./data/train_audio/{label}', label)
    



        

        

100%|██████████| 206/206 [18:02<00:00,  5.26s/it]
